# CIFO Project

In [1]:
import sys
import os

# Add the full path to CIFO_PROJECT to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from copy import deepcopy
import random
from random import sample, shuffle
from statistics import stdev
import pandas as pd


In [2]:
import importlib.util
import sys
import os

# Load solution.py as a module named 'library.solution'
module_path = os.path.abspath("../library/solution.py")
spec = importlib.util.spec_from_file_location("library.solution", module_path)
solution = importlib.util.module_from_spec(spec)
sys.modules["library.solution"] = solution
spec.loader.exec_module(solution)

# Now the import works
from library.solution import Solution

In [3]:
#import players from csv

players = pd.read_csv('../data/players.csv')
print(players)

    Unnamed: 0              Name Position  Skill  Salary (€M)
0            0       Alex Carter       GK     85           90
1            1      Jordan Smith       GK     88          100
2            2     Ryan Mitchell       GK     83           85
3            3    Chris Thompson       GK     80           80
4            4   Blake Henderson       GK     87           95
5            5     Daniel Foster      DEF     90          110
6            6     Lucas Bennett      DEF     85           90
7            7       Owen Parker      DEF     88          100
8            8      Ethan Howard      DEF     80           70
9            9        Mason Reed      DEF     82           75
10          10      Logan Brooks      DEF     86           95
11          11      Caleb Fisher      DEF     84           85
12          12     Nathan Wright      MID     92          120
13          13      Connor Hayes      MID     89          105
14          14      Dylan Morgan      MID     91          115
15      

In [4]:
players.shape

(35, 5)

In [5]:
names = players['Name'].tolist()
positions = players['Position'].tolist()
skills = players['Skill'].tolist()
salaries = players['Salary (€M)'].tolist()

capacity = 750
num_teams=5
team_size=7

## Sports League Optimization Solution

In [6]:
#Creating the Solution for Sports League Optimization (SLO)
from library.solution import Solution
"""Problem definition
Problem: Sports League Optimization (SLO)
Search space: All possible assignments of 35 players into 5 teams, each with 7 players (non-overlapping). Each team must be composed of: 1GK, 2DEF, 2MID, 2FWD.
Representation: List of 5 teams each one with a list of 7 players.
Fitness function: f(x)= standard deviation of the average skill rating of the teams.
Neighborhood: Swap two players with the same position between two teams.
Goal: Minimize f(x).
"""

class SLOSolution(Solution):

    def __init__(
            self,
            names: list[str]=names,
            positions : list[str]=positions,
            skills : list[int]=skills,
            salaries : list[int]=salaries,
            capacity: int = capacity,
            repr=None
    ):
        self.names = names
        self.positions = positions
        self.skills = skills
        self.salaries = salaries
        self.num_teams = num_teams
        self.team_size = team_size
        self.capacity = capacity

        # Grouping players by position
        self.players_by_position = {
            'GK': [i for i, position in enumerate(self.positions) if position == 'GK'],
            'DEF': [i for i, position in enumerate(self.positions) if position == 'DEF'],
            'MID': [i for i, position in enumerate(self.positions) if position == 'MID'],
            'FWD': [i for i, position in enumerate(self.positions) if position == 'FWD'],
        }


        if repr:
            repr = self._validate_repr(repr)

        super().__init__(repr = repr)


    #MAYBE A BETER VALIDATE FUNCTION IS NEEDED
    #--------------------------------------------------------
    def _validate_repr(self,repr):
        # Check if the representation is a list of teams
        if not isinstance(repr, list):
            raise ValueError("Representation must be a list of teams")
        #check if the number of teams is correct
        if len(repr) != self.num_teams:
            raise ValueError("Number of teams must be equal to num_teams")
        #check if each team has the correct number of players
        for team in repr:
            if not isinstance(team, list):
                raise ValueError("Each team must be a list of players")
            if len(team) != self.team_size:
                raise ValueError("Each team must have 7 players")
            

        #check if team is valid (1GK, 2DEF, 2 MID, 2FWD)
        seen_players = set()

        for i, team in enumerate(repr):
            team_positions = {'GK': 0, 'DEF': 0, 'MID': 0, 'FWD': 0}
            team_salary = 0

            seen_players = set()
            for player_idx in team:
                # Check that the player hasn't already been assigned
                if player_idx in seen_players:
                    raise ValueError(f"Player {player_idx} assigned to multiple teams")
                seen_players.add(player_idx)

                pos = self.positions[player_idx]
                sal = self.salaries[player_idx] # TO BE CONFIRMED

                if pos not in team_positions:
                    raise ValueError(f"Invalid position; '{pos}', for player {player_idx}")
                team_positions[pos] += 1 
                team_salary += sal # TO BE CONFIRMED

            
            if team_positions != {'GK': 1, 'DEF': 2, 'MID': 2, 'FWD': 2}:
                raise ValueError(f"Team {i} has invalid composition: {team_positions}")
            
            # TO BE CONFIRMED
            if team_salary > self.capacity:
                raise ValueError(f"Team {i} exceeds salary cap: {team_salary:.2f}M > {self.capacity}M")
        return repr
        
    
    def random_initial_representation(self):
        isValid = False

        while not isValid:
            try:
                repr = []
                used_players = set()

                for _ in range(self.num_teams):
                    team = []
                    gk = random.choice([p for p in self.players_by_position['GK'] if p not in used_players])
                    used_players.add(gk)
                    team.append(gk)

                    def_players = random.sample([p for p in self.players_by_position['DEF'] if p not in used_players], 2)
                    used_players.update(def_players)
                    team.extend(def_players)

                    mid_players = random.sample([p for p in self.players_by_position['MID'] if p not in used_players], 2)
                    used_players.update(mid_players)
                    team.extend(mid_players)

                    fwd_players = random.sample([p for p in self.players_by_position['FWD'] if p not in used_players], 2)
                    used_players.update(fwd_players)
                    team.extend(fwd_players)

                    repr.append(team)

                repr = self._validate_repr(repr)
                isValid = True

            except (ValueError, IndexError) as e:
                print(f"Invalid representation: {e}")
                isValid = False

        return repr

                    
    def fitness(self):

        team_avg_skill=[]

        for team in self.repr:
            team_salary = sum(self.salaries[player_idx] for player_idx in team)
            if team_salary > self.capacity:
                return 999

            team_skills=[self.skills[player_idx] for player_idx in team]
            avg_skill=sum(team_skills)/len(team_skills)
            team_avg_skill.append(avg_skill)
        
        return stdev(team_avg_skill)

In [7]:
#create a team to test the code
slo_solution = SLOSolution(
    names=names, 
    positions=positions, 
    skills=skills, 
    salaries=salaries, 
    capacity=capacity, 
    repr=None
)

random_teams = slo_solution.random_initial_representation()
print("Random teams:") 
for i, team in enumerate(random_teams):
    print(f"Team {i+1}: {[names[player_idx] for player_idx in team]}")
    #check fitness of the repr
print(f"Fitness: {slo_solution.fitness()}") #TO BE CONFIRMED

Invalid representation: Team 0 exceeds salary cap: 765.00M > 750M
Invalid representation: Team 1 exceeds salary cap: 775.00M > 750M
Random teams:
Team 1: ['Chris Thompson', 'Jaxon Griffin', 'Brayden Hughes', 'Nathan Wright', 'Hunter Cooper', 'Julian Scott', 'Xavier Bryant']
Team 2: ['Alex Carter', 'Lucas Bennett', 'Daniel Foster', 'Dominic Bell', 'Bentley Rivera', 'Landon Powell', 'Sebastian Perry']
Team 3: ['Jordan Smith', 'Mason Reed', 'Owen Parker', 'Gavin Richardson', 'Dylan Morgan', 'Chase Murphy', 'Tyler Jenkins']
Team 4: ['Blake Henderson', 'Logan Brooks', 'Caleb Fisher', 'Connor Hayes', 'Spencer Ward', 'Colton Gray', 'Adrian Collins']
Team 5: ['Ryan Mitchell', 'Maxwell Flores', 'Ethan Howard', 'Ashton Phillips', 'Austin Torres', 'Zachary Nelson', 'Elijah Sanders']
Fitness: 1.7035377355158678


## Mutation Functions

In [8]:
def mutation1():
    pass

## Crossover Functions

In [12]:
# ➜  cifo_project/library/algorithms/crossover.py    (for example)

import random
from typing import List, Tuple

TEAM_SIZE   = 7          # 1 GK, 2 DEF, 2 MID, 2 FWD
NUM_TEAMS   = 5
CHROMO_LEN  = TEAM_SIZE * NUM_TEAMS     # 35 genes  (unique player IDs)

def _flatten(repr_: List[List[int]]) -> List[int]:
    """team-of-lists  ➜  single permutation (length = 35)."""
    return [player for team in repr_ for player in team]

def _unflatten(flat: List[int]) -> List[List[int]]:
    """permutation ➜ team-of-lists (preserves GK/DEF/MID/FWD slot order)."""
    return [flat[i:i+TEAM_SIZE] for i in range(0, CHROMO_LEN, TEAM_SIZE)]


def cycle_crossover(p1: List[List[int]],
                    p2: List[List[int]],
                    rng: random.Random | None = None
                   ) -> Tuple[List[List[int]], List[List[int]]]:
    """
    Standard Cycle Crossover (CX) for permutations, adapted to your
    5×7 matrix representation.  Returns two children (same structure).
    """
    if rng is None:
        rng = random

    # --- flatten the parents -------------------------------------------------
    parent1 = _flatten(p1)
    parent2 = _flatten(p2)

    # mapping gene ➜ index for O(1) look-ups in parent1
    idx_in_p1 = {gene: idx for idx, gene in enumerate(parent1)}

    size     = CHROMO_LEN
    visited  = [False] * size
    child1   = [None]  * size
    child2   = [None]  * size
    cycle_no = 0

    while not all(visited):
        start = visited.index(False)          # first unvisited position
        idx   = start
        cycle = []

        # ------ trace one cycle ----------
        while True:
            cycle.append(idx)
            visited[idx] = True
            gene_from_p2 = parent2[idx]
            idx = idx_in_p1[gene_from_p2]     # where that gene sits in P1
            if visited[idx]:
                break

        # ------ copy genes ---------------
        if cycle_no % 2 == 0:                 # even-numbered cycle
            for i in cycle:
                child1[i] = parent1[i]
                child2[i] = parent2[i]
        else:                                 # odd-numbered cycle
            for i in cycle:
                child1[i] = parent2[i]
                child2[i] = parent1[i]

        cycle_no += 1

    # --- rebuild the 5 teams and return --------------------------------------
    return _unflatten(child1), _unflatten(child2)


In [13]:
# --- create two random parents ---------------------------------
p1 = slo_solution.random_initial_representation()
p2 = slo_solution.random_initial_representation()

# --- crossover --------------------------------------------------
childA_repr, childB_repr = cycle_crossover(p1, p2)

# validate & compute fitness
childA = SLOSolution(repr=childA_repr)
childB = SLOSolution(repr=childB_repr)

print("Child-A fitness:", childA.fitness())
print("Child-B fitness:", childB.fitness())


Child-A fitness: 0.888704629428333
Child-B fitness: 1.0121708333667607


## Selection Functions

In [10]:
def tournament_selection(population, k=3):

    # Select a random subset of the population for the tournament
    tournament_solutions = random.sample(population, k)

    best = min(tournament_solutions, key=lambda ind: ind.fitness())

    return deepcopy(best)

## SLO Genetic Algorithm Solution

In [11]:
class SLOGASolution(SLOSolution):
    def __init__(
            self,
            names,
            positions,
            skills,
            salaries,
            capacity,
            mutation_function,
            crossover_function,
            repr=None,
    ):
        super().__init__(
            names=names,
            positions=positions,
            skills=skills,
            salaries=salaries,
            capacity=capacity,
            repr=repr,
        )
        self.mutation_function = mutation_function
        self.crossover_function = crossover_function
        